# Sorghum Flowering Predictions with Ensemble Models

In [1]:
#important libraries
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor,\
StackingRegressor,VotingRegressor,HistGradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error,explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, RepeatedKFold
from matplotlib import pyplot
from scipy.stats import sem
from numpy import mean
from numpy import std

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px

import itertools
from itertools import permutations
pd.set_option("display.max_rows", None, "display.max_columns", None)
sns.set()

%matplotlib inline

## Import Train and Test Data

In [15]:
df_ = pd.read_csv('../../../data/data_red_imputed-1.2.csv',index_col = 0)
df_.head(3)

PlotID    B       CHL     Ca    Cu  DaysToBloom       EWT     Fe     K  \
0    5001  6.4  467.6750  0.674  17.6         83.0  0.048621  171.0  1.93   
1    5002  4.6  512.3625  0.447  12.5         76.0  0.043236  134.0  2.24   
2    5003  5.1  606.6625  0.583  15.4          NaN  0.033848  159.0  1.54   

        LWC     Mg    Mn      N      P         PI  PlantHeight      S  \
0  0.816438  0.245  31.0  3.612  0.381  PI 656057        128.0  0.206   
1  0.768144  0.143  34.0  2.916  0.387  PI 656011        110.0  0.196   
2  0.751805  0.260  36.0  3.749  0.492  PI 576347          NaN  0.196   

         SLA Treatment     X1000     X1001     X1002     X1003     X1004  \
0  91.477861        HN  0.474277  0.474403  0.474529  0.474655  0.474781   
1  76.626487        HN  0.491301  0.491336  0.491371  0.491406  0.491441   
2  89.492182        HN  0.516873  0.517113  0.517353  0.517592  0.517832   

      X1005     X1006     X1007     X1008     X1009     X1010     X1011  \
0  0.474907  0.475033  0.475158  0.475284  0.475410  0.475536  0.475688   
1  0.491476  0.491511  0.491546  0.491581  0.491616  0.491651  0.491807   
2  0.518071  0.518311  0.518551  0.518790  0.519030  0.519270  0.519422   

      X1012     X1013     X1014     X1015     X1016     X1017     X1018  \
0  0.475883  0.476092  0.476295  0.476547  0.476731  0.476871  0.477020   
1  0.491928  0.492030  0.492137  0.492215  0.492333  0.492478  0.492602   
2  0.519596  0.519787  0.519946  0.520070  0.520249  0.520458  0.520623   

      X1019     X1020     X1021     X1022     X1023     X1024     X1025  \
0  0.477147  0.477276  0.477380  0.477523  0.477682  0.477841  0.478011   
1  0.492748  0.492863  0.492936  0.493039  0.493149  0.493257  0.493370   
2  0.520799  0.520966  0.521104  0.521231  0.521360  0.521487  0.521602   

      X1026     X1027     X1028     X1029     X1030     X1031     X1032  \
0  0.478220  0.478387  0.478526  0.478696  0.478880  0.479037  0.479199   
1  0.493498  0.493618  0.493741  0.493842  0.493932  0.494028  0.494113   
2  0.521681  0.521786  0.521905  0.522024  0.522147  0.522257  0.522367   

      X1033     X1034     X1035     X1036     X1037     X1038     X1039  \
0  0.479345  0.479467  0.479586  0.479698  0.479813  0.479914  0.480025   
1  0.494193  0.494277  0.494364  0.494443  0.494519  0.494613  0.494685   
2  0.522483  0.522612  0.522701  0.522747  0.522845  0.522958  0.523044   

      X1040     X1041     X1042     X1043     X1044     X1045     X1046  \
0  0.480153  0.480272  0.480370  0.480458  0.480537  0.480597  0.480662   
1  0.494733  0.494779  0.494834  0.494890  0.494936  0.494995  0.495023   
2  0.523150  0.523272  0.523350  0.523371  0.523421  0.523492  0.523525   

      X1047     X1048     X1049     X1050     X1051     X1052     X1053  \
0  0.480722  0.480787  0.480848  0.480909  0.480994  0.481044  0.481075   
1  0.495011  0.495038  0.495073  0.495094  0.495132  0.495155  0.495170   
2  0.523510  0.523544  0.523576  0.523627  0.523739  0.523807  0.523879   

      X1054     X1055     X1056     X1057     X1058     X1059     X1060  \
0  0.481094  0.481124  0.481161  0.481211  0.481283  0.481314  0.481326   
1  0.495154  0.495154  0.495162  0.495141  0.495147  0.495127  0.495105   
2  0.523900  0.523930  0.523920  0.523904  0.523914  0.523927  0.523941   

      X1061     X1062     X1063     X1064     X1065     X1066     X1067  \
0  0.481318  0.481266  0.481261  0.481272  0.481268  0.481287  0.481282   
1  0.495057  0.494980  0.494923  0.494857  0.494803  0.494767  0.494723   
2  0.523960  0.524011  0.524020  0.524012  0.524002  0.523972  0.523945   

      X1068     X1069     X1070     X1071     X1072     X1073     X1074  \
0  0.481259  0.481224  0.481182  0.481143  0.481075  0.481007  0.480984   
1  0.494677  0.494634  0.494592  0.494530  0.494485  0.494447  0.494378   
2  0.523913  0.523883  0.523884  0.523832  0.523790  0.523789  0.523730   

      X1075     X1076     X1077     X1078     X1079     X1080     X1081  \
0  0.480986

In [12]:
#reflectance data
waves = list(df_.columns)[18:-8]


In [7]:
df = pd.read_csv('../../../data/data_yellow_imputed-1.2.csv',index_col = 0)
df.head()

PlotID    B       CHL     Ca    Cu  DaysToBloom       EWT     Fe     K  \
0    5001  6.4  467.6750  0.674  17.6         83.0  0.048621  171.0  1.93   
1    5002  4.6  512.3625  0.447  12.5         76.0  0.043236  134.0  2.24   
2    5009  5.7  606.9375  0.416  15.8         61.0  0.040890  177.0  1.74   
3    5011  4.9  650.2500  0.607  14.6         65.0  0.046922  162.0  2.02   
4    5014  NaN       NaN    NaN   NaN         83.0       NaN    NaN   NaN   

        LWC     Mg    Mn      N      P         PI  PlantHeight      S  \
0  0.816438  0.245  31.0  3.612  0.381  PI 656057        128.0  0.206   
1  0.768144  0.143  34.0  2.916  0.387  PI 656011        110.0  0.196   
2  0.748609  0.215  36.0  3.333  0.405     BTx623         62.0  0.193   
3  0.781696  0.206  41.0  3.767  0.456  PI 533789         97.0  0.225   
4       NaN    NaN   NaN    NaN    NaN  PI 534117        128.0    NaN   

         SLA Treatment     X1000     X1001     X1002     X1003     X1004  \
0  91.477861        HN  0.474277  0.474403  0.474529  0.474655  0.474781   
1  76.626487        HN  0.491301  0.491336  0.491371  0.491406  0.491441   
2  72.826292        HN  0.519076  0.519423  0.519770  0.520118  0.520465   
3  76.312692        HN  0.505631  0.505661  0.505691  0.505721  0.505751   
4        NaN        HN  0.493152  0.493415  0.493678  0.493941  0.494204   

      X1005     X1006     X1007     X1008     X1009     X1010     X1011  \
0  0.474907  0.475033  0.475158  0.475284  0.475410  0.475536  0.475688   
1  0.491476  0.491511  0.491546  0.491581  0.491616  0.491651  0.491807   
2  0.520812  0.521159  0.521506  0.521853  0.522200  0.522547  0.522712   
3  0.505780  0.505810  0.505840  0.505870  0.505900  0.505930  0.506092   
4  0.494467  0.494730  0.494993  0.495256  0.495519  0.495782  0.495883   

      X1012     X1013     X1014     X1015     X1016     X1017     X1018  \
0  0.475883  0.476092  0.476295  0.476547  0.476731  0.476871  0.477020   
1  0.491928  0.492030  0.492137  0.492215  0.492333  0.492478  0.492602   
2  0.522885  0.523075  0.523255  0.523439  0.523596  0.523754  0.523889   
3  0.506280  0.506488  0.506657  0.506820  0.506995  0.507132  0.507306   
4  0.496025  0.496162  0.496318  0.496518  0.496696  0.496891  0.497026   

      X1019     X1020     X1021     X1022     X1023     X1024     X1025  \
0  0.477147  0.477276  0.477380  0.477523  0.477682  0.477841  0.478011   
1  0.492748  0.492863  0.492936  0.493039  0.493149  0.493257  0.493370   
2  0.523965  0.524094  0.524232  0.524383  0.524581  0.524749  0.524909   
3  0.507507  0.507722  0.507975  0.508183  0.508402  0.508599  0.508775   
4  0.497128  0.497249  0.497315  0.497422  0.497597  0.497720  0.497826   

      X1026     X1027     X1028     X1029     X1030     X1031     X1032  \
0  0.478220  0.478387  0.478526  0.478696  0.478880  0.479037  0.479199   
1  0.493498  0.493618  0.493741  0.493842  0.493932  0.494028  0.494113   
2  0.525081  0.525195  0.525268  0.525364  0.525423  0.525521  0.525639   
3  0.508904  0.509052  0.509203  0.509341  0.509463  0.509589  0.509693   
4  0.497904  0.498022  0.498129  0.498252  0.498399  0.498550  0.498723   

      X1033     X1034     X1035     X1036     X1037     X1038     X1039  \
0  0.479345  0.479467  0.479586  0.479698  0.479813  0.479914  0.480025   
1  0.494193  0.494277  0.494364  0.494443  0.494519  0.494613  0.494685   
2  0.525765  0.525923  0.526042  0.526136  0.526260  0.526391  0.526501   
3  0.509838  0.510027  0.510163  0.510307  0.510432  0.510509  0.510592   
4  0.498851  0.498989  0.499096  0.499132  0.499209  0.499300  0.499389   

      X1040     X1041     X1042     X1043     X1044     X1045     X1046  \
0  0.480153  0.480272  0.480370  0.480458  0.480537  0.480597  0.480662   
1  0.494733  0.494779  0.494834  0.494890  0.494936  0.494995  0.495023   
2  0.526616  0.526742  0.526853  0.526945  0.527013  0.527071  0.527143   
3  0.510681  0.510755  0.510843  0.510924  0.511011  0.511128  0.511195   
4  0.499499  0

In [8]:
df.shape

(1294, 2172)

In [9]:
df = df[~df['X1000'].isnull()]
df.shape


(817, 2172)

In [16]:
df =df[df_.columns]
df.head()

PlotID    B       CHL     Ca    Cu  DaysToBloom       EWT     Fe     K  \
0    5001  6.4  467.6750  0.674  17.6         83.0  0.048621  171.0  1.93   
1    5002  4.6  512.3625  0.447  12.5         76.0  0.043236  134.0  2.24   
2    5009  5.7  606.9375  0.416  15.8         61.0  0.040890  177.0  1.74   
3    5011  4.9  650.2500  0.607  14.6         65.0  0.046922  162.0  2.02   
4    5014  NaN       NaN    NaN   NaN         83.0       NaN    NaN   NaN   

        LWC     Mg    Mn      N      P         PI  PlantHeight      S  \
0  0.816438  0.245  31.0  3.612  0.381  PI 656057        128.0  0.206   
1  0.768144  0.143  34.0  2.916  0.387  PI 656011        110.0  0.196   
2  0.748609  0.215  36.0  3.333  0.405     BTx623         62.0  0.193   
3  0.781696  0.206  41.0  3.767  0.456  PI 533789         97.0  0.225   
4       NaN    NaN   NaN    NaN    NaN  PI 534117        128.0    NaN   

         SLA Treatment     X1000     X1001     X1002     X1003     X1004  \
0  91.477861        HN  0.474277  0.474403  0.474529  0.474655  0.474781   
1  76.626487        HN  0.491301  0.491336  0.491371  0.491406  0.491441   
2  72.826292        HN  0.519076  0.519423  0.519770  0.520118  0.520465   
3  76.312692        HN  0.505631  0.505661  0.505691  0.505721  0.505751   
4        NaN        HN  0.493152  0.493415  0.493678  0.493941  0.494204   

      X1005     X1006     X1007     X1008     X1009     X1010     X1011  \
0  0.474907  0.475033  0.475158  0.475284  0.475410  0.475536  0.475688   
1  0.491476  0.491511  0.491546  0.491581  0.491616  0.491651  0.491807   
2  0.520812  0.521159  0.521506  0.521853  0.522200  0.522547  0.522712   
3  0.505780  0.505810  0.505840  0.505870  0.505900  0.505930  0.506092   
4  0.494467  0.494730  0.494993  0.495256  0.495519  0.495782  0.495883   

      X1012     X1013     X1014     X1015     X1016     X1017     X1018  \
0  0.475883  0.476092  0.476295  0.476547  0.476731  0.476871  0.477020   
1  0.491928  0.492030  0.492137  0.492215  0.492333  0.492478  0.492602   
2  0.522885  0.523075  0.523255  0.523439  0.523596  0.523754  0.523889   
3  0.506280  0.506488  0.506657  0.506820  0.506995  0.507132  0.507306   
4  0.496025  0.496162  0.496318  0.496518  0.496696  0.496891  0.497026   

      X1019     X1020     X1021     X1022     X1023     X1024     X1025  \
0  0.477147  0.477276  0.477380  0.477523  0.477682  0.477841  0.478011   
1  0.492748  0.492863  0.492936  0.493039  0.493149  0.493257  0.493370   
2  0.523965  0.524094  0.524232  0.524383  0.524581  0.524749  0.524909   
3  0.507507  0.507722  0.507975  0.508183  0.508402  0.508599  0.508775   
4  0.497128  0.497249  0.497315  0.497422  0.497597  0.497720  0.497826   

      X1026     X1027     X1028     X1029     X1030     X1031     X1032  \
0  0.478220  0.478387  0.478526  0.478696  0.478880  0.479037  0.479199   
1  0.493498  0.493618  0.493741  0.493842  0.493932  0.494028  0.494113   
2  0.525081  0.525195  0.525268  0.525364  0.525423  0.525521  0.525639   
3  0.508904  0.509052  0.509203  0.509341  0.509463  0.509589  0.509693   
4  0.497904  0.498022  0.498129  0.498252  0.498399  0.498550  0.498723   

      X1033     X1034     X1035     X1036     X1037     X1038     X1039  \
0  0.479345  0.479467  0.479586  0.479698  0.479813  0.479914  0.480025   
1  0.494193  0.494277  0.494364  0.494443  0.494519  0.494613  0.494685   
2  0.525765  0.525923  0.526042  0.526136  0.526260  0.526391  0.526501   
3  0.509838  0.510027  0.510163  0.510307  0.510432  0.510509  0.510592   
4  0.498851  0.498989  0.499096  0.499132  0.499209  0.499300  0.499389   

      X1040     X1041     X1042     X1043     X1044     X1045     X1046  \
0  0.480153  0.480272  0.480370  0.480458  0.480537  0.480597  0.480662   
1  0.494733  0.494779  0.494834  0.494890  0.494936  0.494995  0.495023   
2  0.526616  0.526742  0.526853  0.526945  0.527013  0.527071  0.527143   
3  0.510681  0.510755  0.510843  0.510924  0.511011  0.511128  0.511195   
4  0.499499  0

### Calculate Principal Components

In [ ]:
sub = df[df.columns[18:-8]] #reflectance data
sub.head()

In [ ]:
n_components =50
pca = PCA(n_components)
sub_trans = pca.fit_transform(sub.values)
sub_df = pd.DataFrame(data = sub_trans
         , columns = ["PC"+str(i) for i in list(range(1,n_components+1))])
sub_df.head()

In [ ]:
total_var = pca.explained_variance_ratio_.sum() * 100
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

plt.figure(figsize = (15,8))
sns.set_context('paper')
sns.barplot(x = list(range(1,n_components+1,1)), y = exp_var_cumul)

In [ ]:
total_var #explained variance

In [ ]:
sub_df['yield'] = df['yield']
sub_df['treatment'] = df.Treatment

In [ ]:
sub_df.head()

In [ ]:
sub_df.shape

### Train Test Split

In [ ]:
#df.rename(columns = {'Treatment':'treatment'},inplace = True)

In [ ]:
## Data Split
X = sub_df[sub_df.columns[:-2]]
y = sub_df[['treatment','DaysToBloom']]
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30)

train_data = pd.concat([X_train, y_train], axis = 1)
print(f"Train Dataset: {train_data.shape[0]} row/records. F: {train_data[train_data.treatment=='HN'].shape[0]} and N: {train_data[train_data.treatment =='LN'].shape[0]}\n")

test_data = pd.concat([X_test,y_test], axis = 1)
print(f"Train Dataset: {test_data.shape[0]} row/records. F: {test_data[test_data.treatment=='HN'].shape[0]} and N: {test_data[test_data.treatment =='LN'].shape[0]}")



In [ ]:
#features
features = list(sub_df.columns)[:-2]

## Important Functions

In [ ]:
# evaluate a model with a given number of repeats
def evaluate_model(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    estimators= 100
    model = RandomForestRegressor(n_estimators = estimators)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)
    return scores

## RF  - Both Treatment

In [ ]:
#check the head of training dataset
train_data.head()

In [ ]:
sub_df.head()

In [ ]:
## Initialize Models with default parameters
X = sub_df[list(sub_df.columns)[:-2]]
y = sub_df['DaysToBloom']
results = list()
for i in range(1,6):
    scores = evaluate_model(X,y,i)
    # summarize
    print('>%d mean=%.4f se=%.3f' % (i, mean(scores), sem(scores)))
    # store
    results.append(scores)

# cv = KFold(n_splits=10, random_state=1, shuffle=True)
# # evaluate model
# scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)
# scores

In [ ]:
plt.figure(figsize = (15,8))
pyplot.boxplot(results, labels=[str(r) for r in range(1,6)], showmeans=True)
plt.ylabel("r2")
plt.xlabel("K-fold Validation")
plt.title("RF - HN/LN Nitrogen Only")
pyplot.show()

## RF - Full Nitrogen


In [ ]:
X = sub_df[sub_df.treatment =="HN"][list(sub_df.columns)[:-2]]
y = sub_df[sub_df.treatment =="HN"]['DaysToBloom']
results = list()
for i in range(1,6):
    scores = evaluate_model(X,y,i)
    # summarize
    print('>%d mean=%.4f se=%.3f' % (i, mean(scores), sem(scores)))
    # store
    results.append(scores)

# cv = KFold(n_splits=10, random_sta

In [ ]:
plt.figure(figsize = (15,8))
pyplot.boxplot(results, labels=[str(r) for r in range(1,6)], showmeans=True)
plt.ylabel("r2")
plt.xlabel("K-fold Validation")
plt.title("RF - High Nitrogen Only")
pyplot.show()

## RF - Nitrogen Deprived

In [ ]:
X = sub_df[sub_df.treatment =="LN"][list(sub_df.columns)[:-2]]
y = sub_df[sub_df.treatment =="LN"]['DaysToBloom']
results = list()
for i in range(1,6):
    scores = evaluate_model(X,y,i)
    # summarize
    print('>%d mean=%.4f se=%.3f' % (i, mean(scores), sem(scores)))
    # store
    results.append(scores)

# cv = KFold(n_splits=10, random_sta

In [ ]:
plt.figure(figsize = (15,8))
pyplot.boxplot(results, labels=[str(r) for r in range(1,6)], showmeans=True)
plt.ylabel("r2")
plt.xlabel("K-fold Validation")
plt.title("RF - Low Nitrogen Only")
pyplot.show()